In [9]:
import torch
import vtk
import matplotlib.pyplot as plt

In [15]:
from torch.utils.data import Dataset
class Mesh(Dataset):
    def __init__(self):
        self.coordinates = None
        self.faces = None

class Polygon(Dataset):
    def __init__(self, num_node, num_face):
        self.nodeID = torch.zeros(num_node)
        self.coordinates = torch.zeros(num_node, 2)
        self.faces = torch.zeros(num_face, 3)

def create_mesh_data(vtk_file_path):
    reader = vtk.vtkDataSetReader()
    reader.SetFileName(vtk_file_path)
    reader.Update()

    dataset = reader.GetOutput()

    mesh = Mesh()
    # 座標情報を取得
    points = dataset.GetPoints()
    num_points = points.GetNumberOfPoints()
    coordinates = torch.zeros(num_points, 3)
    for i in range(num_points):
        coordinates[i] = torch.tensor(points.GetPoint(i))

    mesh.coordinates = coordinates[:, :2]                        # mesh.coordinates を定義

    # 面情報を取得
    polys = dataset.GetPolys()
    num_polys = polys.GetNumberOfCells()
    mesh.faces = torch.zeros(num_polys, 3, dtype=int)           # mesh.faces を定義

    # 各三角形の情報を取得
    polys.InitTraversal()
    for i in range(num_polys):
        cell = vtk.vtkIdList()
        if polys.GetNextCell(cell) == 0:
            break
        mesh.faces[i] = torch.tensor([cell.GetId(0), cell.GetId(1), cell.GetId(2)])
        
    # 各セルの各辺の隣接セル数を調べる
    edge_neighbors = {}
    num_cells = dataset.GetNumberOfCells()
    for cell_index in range(num_cells):
        cell = dataset.GetCell(cell_index)
        num_edges = cell.GetNumberOfEdges()

        for edge_index in range(num_edges):
            edge = cell.GetEdge(edge_index)
            edge_points = edge.GetPointIds()

            # 辺を構成する点のインデックスを取得
            point1_id = edge_points.GetId(0)
            point2_id = edge_points.GetId(1)

            # 辺を構成する点のインデックスを照準にソート
            edge_key = (min(point1_id, point2_id), max(point1_id, point2_id))

            # 辺の隣接セル数をカウント
            if edge_key in edge_neighbors:
                edge_neighbors[edge_key] += 1
            else:
                edge_neighbors[edge_key] = 1 

    boundary_edges = []
    # 境界上の辺を特定
    for edge_key, num_neighbors in edge_neighbors.items():
        if num_neighbors == 1:
            boundary_edges.append(edge_key)

    # 境界上の辺を構成する頂点の番号を取得
    boundary_points = set()     # 集合を表すデータ型、順番を持たず、重複した要素は取り除かれる

    for edge_key in boundary_edges:
        boundary_points.add(edge_key[0])
        boundary_points.add(edge_key[1])
    poly_count = 0
    for pointId in range(num_points):       # pointId:自由点の頂点番号
        if pointId in boundary_points:
            continue
        else:
            poly_count += 1
            # print("pointId:", pointId)
        mask = (mesh.faces == pointId)
        if mask.any():
            count = torch.sum(mask).item()
        num_node = count + 1
        num_face = count
        polygon_number = poly_count - 1 



        
        polygon_i = f"polygon_{polygon_number}"
        # print(polygon_i)
        polygon_i = Polygon(num_node, num_face)
        
        element_to_check = pointId
        polygon_i.face = mesh.faces[(mesh.faces == element_to_check).any(dim=1)]
        # print(polygon_i.face)

        polygon_i.nodeId = set()
        for i in range(len(polygon_i.face)):
            polygon_i.nodeId.add(polygon_i.face[i, 0].item())
            polygon_i.nodeId.add(polygon_i.face[i, 1].item())
            polygon_i.nodeId.add(polygon_i.face[i, 2].item())
        sorted_nodeId = sorted(polygon_i.nodeId)
        polygon_i.nodeID = torch.tensor(list(sorted_nodeId))
        
        point_id_index = (polygon_i.nodeID == pointId).nonzero().item()

        value_to_move = polygon_i.nodeID[point_id_index]
        polygon_i.nodeID = torch.cat((value_to_move.unsqueeze(0), polygon_i.nodeID[polygon_i.nodeID != pointId]))

        # print(polygon_i.nodeID)

        for i in range(num_node):
            polygon_i.coordinates[i] = mesh.coordinates[polygon_i.nodeID[i].item()]


        indices = torch.nonzero(torch.isin(polygon_i.face, polygon_i.nodeID))
        for idx in range(indices.size(0)):
            row_idx, col_idx = indices[idx]
            value_to_replace = polygon_i.face[row_idx, col_idx]
            polygon_i.face[row_idx, col_idx] = (polygon_i.nodeID == value_to_replace).nonzero().item()
        polygon_i.faces = polygon_i.face.long()

        setattr(mesh, f"polygon_{polygon_number}", polygon_i)
    return mesh


    


In [11]:
# vtk_file_path = "./men_re.vtk"
# mesh = create_mesh_data(vtk_file_path)
# print("mesh.coordinates:", mesh.coordinates)
# print("mesh.faces:", mesh.faces)
# print("mesh.polygon_0.nodeID:", mesh.polygon_0.nodeID)

# フォルダ内のすべての.vtkファイルに順に実行するようにする

In [16]:
class Dataset(Dataset):
    def __init__(self, num_files):
        None
    

In [17]:
import glob

# フォルダ内のすべてのvtkファイルにアクセスする
vtk_files = glob.glob("./*.vtk")

num_vtk_files = len(vtk_files)
dataset = Dataset(num_vtk_files)

# ファイルに順にアクセスする
for i in range(num_vtk_files):
    print("File Name:", vtk_files[i])
    mesh = create_mesh_data(vtk_files[i])
    setattr(dataset, f"mesh_{i}", mesh)

print(dataset.mesh_0.polygon_0.coordinates)
print(dataset.mesh_1.polygon_0.coordinates)

File Name: ./men.vtk
tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2],
        [3, 0],
        [3, 1],
        [3, 2],
        [4, 0],
        [4, 1],
        [4, 2],
        [5, 0],
        [5, 1],
        [5, 2]])
tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2],
        [3, 0],
        [3, 1],
        [3, 2],
        [4, 0],
        [4, 1],
        [4, 2],
        [5, 0],
        [5, 1],
        [5, 2]])
tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2],
        [3, 0],
        [3, 1],
        [3, 2],
        [4, 0],
        [4, 1],
        [4, 2],
        [5, 0],
        [5, 1],
        [5, 2]])
tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],


In [14]:
print(dataset.mesh_1.polygon_0.nodeID)
print(dataset.mesh_1.polygon_0.faces)

tensor([ 200,  161,  162,  304,  305, 2347, 2364])
tensor([[1, 2, 0],
        [1, 0, 3],
        [0, 2, 4],
        [5, 6, 0],
        [5, 0, 4],
        [6, 3, 0]])
